In [ ]:
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import zipfile


In [13]:
df_train = pd.read_csv(r"C:\Users\John\Desktop\Patent Datasets\final_data\Google patent phrase similarity\train.csv")
df_test = pd.read_csv(r"C:\Users\John\Desktop\Patent Datasets\final_data\Google patent phrase similarity\test.csv")
df_train_us = pd.read_csv(r"C:\Users\John\Desktop\Patent Datasets\final_data\USPTO patent match\train.csv")
df_test_us = pd.read_csv(r"C:\Users\John\Desktop\Patent Datasets\final_data\USPTO patent match\test.csv")

In [14]:
df_train = df_train[df_train['context'] == 'F03']
df_test = df_test[df_test['context'] == 'F03']
df_train_us = df_train_us[df_train_us['context'] == 'F03']
#df_test_us = df_test_us[df_test_us['context'] == 'F03']

In [17]:
# Ensure that only necessary columns are selected
df_train = df_train[["anchor", "target", "score"]]
df_test = df_test[["anchor", "target", "score"]]
df_train_us = df_train_us[["anchor", "target", "score"]]
#df_test_us = df_test_us[["anchor", "target", "score"]]
df_train = pd.concat([df_train, df_train_us], ignore_index=True)
#df_test.append(df_test_us)
# Convert DataFrame to Hugging Face Dataset
dataset_train = Dataset.from_pandas(df_train)
dataset_test = Dataset.from_pandas(df_test)

In [ ]:


# Load the tokenizer
model_name = "princeton-nlp/sup-simcse-bert-base-uncased"  # Change if using unsupervised
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the tokenization function
def preprocess_function(examples):
    # Tokenize the sentence pairs
    inputs = tokenizer(
        examples["anchor"], 
        examples["target"], 
        truncation=True, 
        padding="max_length", 
        max_length=32
    )
    # Include the similarity score as labels
    inputs["labels"] = examples["score"]
    return inputs

# Apply tokenization to the datasets
tokenized_train_dataset = dataset_train.map(preprocess_function, batched=True)
tokenized_test_dataset = dataset_test.map(preprocess_function, batched=True)


Map:   0%|          | 0/446 [00:00<?, ? examples/s]

Map:   0%|          | 0/27 [00:00<?, ? examples/s]

In [ ]:


# Load the SimCSE model configured for regression
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1  # Regression requires one output label
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at princeton-nlp/sup-simcse-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:


# Define training arguments
training_args = TrainingArguments(
    output_dir="./simcse_finetuned",
    evaluation_strategy="epoch",  # Evaluate after every epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",  # Save model after every epoch
    logging_dir="./logs",
    logging_steps=50,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
)



C:\Users\John\AppData\Local\Temp\ipykernel_25128\2026107613.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
trainer.train()

  0%|          | 0/84 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.08301157504320145, 'eval_runtime': 0.0261, 'eval_samples_per_second': 1033.183, 'eval_steps_per_second': 76.532, 'epoch': 1.0}
{'loss': 0.0646, 'grad_norm': 1.4849673509597778, 'learning_rate': 8.095238095238097e-06, 'epoch': 1.79}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.07862962782382965, 'eval_runtime': 0.0265, 'eval_samples_per_second': 1017.76, 'eval_steps_per_second': 75.39, 'epoch': 2.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.0769670382142067, 'eval_runtime': 0.0186, 'eval_samples_per_second': 1450.331, 'eval_steps_per_second': 107.432, 'epoch': 3.0}
{'train_runtime': 7.2402, 'train_samples_per_second': 184.802, 'train_steps_per_second': 11.602, 'train_loss': 0.04997556550162179, 'epoch': 3.0}


TrainOutput(global_step=84, training_loss=0.04997556550162179, metrics={'train_runtime': 7.2402, 'train_samples_per_second': 184.802, 'train_steps_per_second': 11.602, 'total_flos': 22002464451456.0, 'train_loss': 0.04997556550162179, 'epoch': 3.0})

In [26]:
df_val = pd.read_csv(r"C:\Users\John\Desktop\Patent Datasets\final_data\Google patent phrase similarity\validation.csv")
df_val = df_val[df_val['context'] == 'F03']
# Ensure only necessary columns are selected
df_val = df_val[["anchor", "target", "score"]]

# Convert the validation DataFrame to a Hugging Face Dataset
dataset_val = Dataset.from_pandas(df_val)

In [27]:
tokenized_val_dataset = dataset_val.map(preprocess_function, batched=True)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [32]:
# Evaluate the model on the test dataset
test_results = trainer.evaluate(eval_dataset=tokenized_val_dataset)
#test_results = trainer.evaluate(eval_dataset=tokenized_test_dataset)
print("Test results:", test_results)

  0%|          | 0/1 [00:00<?, ?it/s]

Test results: {'eval_loss': 0.06196527183055878, 'eval_runtime': 0.0976, 'eval_samples_per_second': 10.245, 'eval_steps_per_second': 10.245, 'epoch': 3.0}


In [38]:
# Save the model
model.save_pretrained('./saved_model_ony_phrases')
tokenizer.save_pretrained('./saved_model_ony_phrases')

('./saved_model_ony_phrases\\tokenizer_config.json',
 './saved_model_ony_phrases\\special_tokens_map.json',
 './saved_model_ony_phrases\\vocab.txt',
 './saved_model_ony_phrases\\added_tokens.json',
 './saved_model_ony_phrases\\tokenizer.json')

## Second Fine Tune

In [49]:

# Path to the zip file
zip_file_path = r'C:\Users\John\Desktop\Patent Datasets\final_data\HiDrive\patentmatch_train_balanced.zip'
zip_file_path_test = r'C:\Users\John\Desktop\Patent Datasets\final_data\HiDrive\patentmatch_test_balanced.zip'

# Open the zip file
with zipfile.ZipFile(zip_file_path, 'r') as z:
    # List all files in the zip
    print(z.namelist())
    
    # Assuming the zip contains a TSV file named 'data.tsv'
    with z.open('patentmatch_train_balanced.tsv') as f:
        df_train_cites = pd.read_csv(f, delimiter='\t')

# Open the zip file
with zipfile.ZipFile(zip_file_path_test, 'r') as z:
    # List all files in the zip
    print(z.namelist())
    
    # Assuming the zip contains a TSV file named 'data.tsv'
    with z.open('patentmatch_test_balanced.tsv') as f:
        df_test_cites = pd.read_csv(f, delimiter='\t')



['patentmatch_train_balanced.tsv']
['patentmatch_test_balanced.tsv']


In [55]:
# Calculate the average number of words in the 'text' column
average_words = df_test_cites['text_b'].apply(lambda x: len(str(x).split())).mean()

print(f"Average number of words: {average_words}")

Average number of words: 93.42234391169369


In [51]:
# Ensure that only necessary columns are selected
df_train_ct = df_train_cites[["text", "text_b", "label"]]
df_test_ct = df_test_cites[["text", "text_b", "label"]]

# Convert DataFrame to Hugging Face Dataset
dataset_train_ct = Dataset.from_pandas(df_train_ct)
dataset_test_ct = Dataset.from_pandas(df_test_ct)

In [52]:
# Load the saved model and tokenizer
model_name = './saved_model_ony_phrases'
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [77]:
# Define the tokenization function
def preprocess_function(examples):
    # Tokenize the sentence pairs
    inputs = tokenizer(
        examples["text"], 
        examples["text_b"], 
        truncation=True, 
        padding="max_length", 
        max_length=32
    )
    # Include the similarity score as labels
    inputs["labels"] = [float(label) for label in examples["label"]]
    return inputs

# Apply tokenization to the datasets
tokenized_train_dataset_ct = dataset_train_ct.map(preprocess_function, batched=True)
tokenized_test_dataset_ct = dataset_test_ct.map(preprocess_function, batched=True)

Map:   0%|          | 0/278304 [00:00<?, ? examples/s]

Map:   0%|          | 0/69576 [00:00<?, ? examples/s]

In [80]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./simcse_finetuned",
    evaluation_strategy="epoch",  # Evaluate after every epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",  # Save model after every epoch
    logging_dir="./logs",
    logging_steps=50,
)


# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset_ct,
    eval_dataset=tokenized_test_dataset_ct,
    tokenizer=tokenizer,
)

C:\Users\John\AppData\Local\Temp\ipykernel_25128\1204420052.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [81]:

# Fine-tune the model again
trainer.train()

  0%|          | 0/52182 [00:00<?, ?it/s]

RuntimeError: Found dtype Long but expected Float